<font color="#4b76b7">To start practicing, you will need to make a copy of it. Go to File > Save a Copy in Drive. You can then use the new copy that will appear in the new tab.</font>


# AfterWork Data Science: Getting Started with NLP Project

### Prerequisites

In [1]:
# Importing the required libraries
# ---
# 
import pandas as pd # library for data manipulation
import numpy as np  # librariy for scientific computations
import re           # regex library to perform text preprocessing
import string       # library to work with strings
import nltk         # library for natural language processing
import scipy        # scientific conputing 

# see entire column content in the dataframe
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None) 

### 1. Importing our Data

In [2]:
# Question: Given a new tweets, create a sentiment analysis model that will 
# predict whether a tweet will contain positive or negative sentiment.
# ---
# Dataset url = https://bit.ly/31kqByD 
# ---
#
df = pd.read_csv('https://bit.ly/31kqByD', encoding='latin-1')
df.head()

,Unnamed: 0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the civilized world - http://tinyurl.com/pqcops . And he didn't even drop in for a cup of tea
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,"Had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat and pray!!!!
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #SYTYCD
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


### 2. Data Exploration

In [3]:
# We can determine the size of our dataset
# ---
#
df.shape

(10000, 7)

Seems this dataset will need some data cleaning i.e. columns. We also don't need some columns to perform create our model. We will drop those columns.

### 3. Data Preparation

#### Basic Data Cleaning Techniques

In [4]:
# We rename the columns for ease of referencing our columns later on
# ---
#
df.columns = ['id', 'target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

,id,target,t_id,created_at,query,user,text
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the civilized world - http://tinyurl.com/pqcops . And he didn't even drop in for a cup of tea
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,"Had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat and pray!!!!
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #SYTYCD
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


In [5]:
# We retain the relevant columns by dropping the columns we don't need 
# for creating a sentiment analysis model. 
# ---
#
df = df.drop(['id', 't_id', 'created_at', 'query', 'user'], axis = 1)
df.head()

,target,text
0,0,Obama forges his Muslim alliance against the civilized world - http://tinyurl.com/pqcops . And he didn't even drop in for a cup of tea
1,4,"Had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,0,I am overwhelmed today taking a moment to eat and pray!!!!
3,0,@lindork Tres sad. I was totally a Max fan. #SYTYCD
4,0,"Crap, I was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


In [6]:
# Understanding the distribution of target
# ---
#
df.target.value_counts() 

0    5067
4    4933
Name: target, dtype: int64

In [7]:
# Let's determine whether our columns have the right data types
# ---
#
df.dtypes

target     int64
text      object
dtype: object

In [8]:
# What values are in our target variable?
# ---
#
df.target.unique()

array([0, 4])

These are the two classes to which each document (text) belongs. The target value 0 means a text with a negative sentiment, while that of 4 means a text with a positive sentiment. 

In [9]:
# Let's check for missing values 
# ---
# 
df.isnull().sum()

target    0
text      0
dtype: int64

We don't have any missing values, so we are good to go.

#### Text Processing

In [10]:
# Text Cleaning: Removing all urls/links
# ---
# 
df['text'] =  df['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','', str(x)))
df[['text']].head()

,text
0,Obama forges his Muslim alliance against the civilized world - . And he didn't even drop in for a cup of tea
1,"Had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,I am overwhelmed today taking a moment to eat and pray!!!!
3,@lindork Tres sad. I was totally a Max fan. #SYTYCD
4,"Crap, I was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


In [11]:
# Text Cleaning: Removing @ and # characters or replace them with space
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('#','')
df['text'] = df.text.str.replace('@','')
df.head()

,target,text
0,0,Obama forges his Muslim alliance against the civilized world - . And he didn't even drop in for a cup of tea
1,4,"Had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,0,I am overwhelmed today taking a moment to eat and pray!!!!
3,0,lindork Tres sad. I was totally a Max fan. SYTYCD
4,0,"Crap, I was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


In [12]:
# Text Cleaning: Conversion to lowercase
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df['text'] .str.lower()
df.head()

,target,text
0,0,obama forges his muslim alliance against the civilized world - . and he didn't even drop in for a cup of tea
1,4,"had the most spectacular prom ever but now my bed is serenading me and i must answer, sweet dreams my friends what a wonderful day"
2,0,i am overwhelmed today taking a moment to eat and pray!!!!
3,0,lindork tres sad. i was totally a max fan. sytycd
4,0,"crap, i was counting down the hours until my dad could come home &amp; help me watch my son but now he said is going out to dinner first"


In [13]:
# Text Cleaning: Splitting concatenated words
# ---
# Performing this step will take few minutes...
# ---
# YOUR CODE GOES BELOW
# 

# Installing wordnija and textblob
# ---
#
!pip3 install wordninja
!pip3 install textblob


# Importing those libraries
# ---
#
import wordninja 
from textblob import TextBlob

     |████████████████████████████████| 541 kB 14.5 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=19b4df430867a1389ba7a184b4b41f0863e63ab6785f36d4a7536dcb2ac108b6
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [14]:
# Performing the split
# ---
#
df['text'] = df.text.apply(lambda x: wordninja.split(str(TextBlob(x))))  
df['text'] = df.text.str.join(' ')
df.sample(5) 


,target,text
7810,0,nick carter ey boy i can t see the video because youtube says me the vid is private why i want to see it be sos grandes gua po
4204,0,a ww harry that is so special that they bring harry kal as ' gear on the road with them thought they only hung it at c bp
8977,0,neil hasn't called yet and his phone isn't working getting worried
6262,0,it so tired that im crying for no reason at all im about to try to get an hour and a half in for tonight half what i got last night
7849,0,kiss ability have to wait for some hot water first but that s a great idea since the sun just turned into rain


In [15]:
# Text Cleaning: Removing punctuation characters
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df.text.str.replace('[^\w\s]','')
df.head() 

,target,text
0,0,obama forges his muslim alliance against the civilized world and he didnt even drop in for a cup of tea
1,4,had the most spectacular prom ever but now my bed is serenading me and i must answer sweet dreams my friends what a wonderful day
2,0,i am overwhelmed today taking a moment to eat and pray
3,0,lin dork tres sad i was totally a max fan sytycd
4,0,crap i was counting down the hours until my dad could come home amp help me watch my son but now he said is going out to dinner first


In [16]:
# Text Cleaning: Removing stop words
# ---
# YOUR CODE GOES BELOW
# 
import nltk
nltk.download('stopwords')

# We will first import a list of stopwords in english
# ---
# 
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [17]:
# Removing these Stop Words
# ---
#
df['text'] = df.text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df.head()

,target,text
0,0,obama forges muslim alliance civilized world didnt even drop cup tea
1,4,spectacular prom ever bed serenading must answer sweet dreams friends wonderful day
2,0,overwhelmed today taking moment eat pray
3,0,lin dork tres sad totally max fan sytycd
4,0,crap counting hours dad could come home amp help watch son said going dinner first


In [18]:
# Text Cleaning: Lemmatization
# ---
# YOUR CODE GOES BELOW
#

# For lemmatization, we will need to download wordnet
#
nltk.download('wordnet')

# To perform lemmatization, we use will also import the Word object from the textblob library 
# and pass it the word that you want to lemmatize and then call the lemmatize method as shown
# ---
#
from textblob import Word


# Lemmatizing our text
# ---
#

# Then perform lematization
# ---
#
df['text'] = df.text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,target,text
0,0,obama forge muslim alliance civilized world didnt even drop cup tea
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day
2,0,overwhelmed today taking moment eat pray
3,0,lin dork tres sad totally max fan sytycd
4,0,crap counting hour dad could come home amp help watch son said going dinner first


We won't remove numerics because we could loose meaning of our text if we lost the numerics. We could also further prepare our text by performing spelling correction but this is a resource intensive process that we will skip for now.

#### Feature Engineering Techniques 

In [19]:
# Feature Construction: Length of tweet
# ---
# YOUR CODE GOES BELOW
#

df['tweet_length'] = df['text'].apply(len)
df.head()

,target,text,tweet_length
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81
2,0,overwhelmed today taking moment eat pray,40
3,0,lin dork tres sad totally max fan sytycd,40
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81


In [20]:
# Feature Construction: Word count 
# ---
# YOUR CODE GOES BELOW
# 
df['word_count'] = df.text.apply(lambda x: len(str(x).split(" ")))
df.head()

,target,text,tweet_length,word_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12
2,0,overwhelmed today taking moment eat pray,40,6
3,0,lin dork tres sad totally max fan sytycd,40,8
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15


In [21]:
len(df['text'])

10000

In [22]:
def avg_word(sentence):
  words = sentence.split()
  try:
    z = (sum(len(word) for word in words)/len(words))
  except ZeroDivisionError:
    z = 0 
  return z

In [23]:
# Feature Construction: Word density (Average no. of words / tweet)
# ---
# YOUR CODE GOES BELOW
#
df['word_density'] = df.text.apply(lambda x: avg_word(x))
df.head()

,target,text,tweet_length,word_count,word_density
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333
2,0,overwhelmed today taking moment eat pray,40,6,5.833333
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667


In [24]:
# Feature Construction: Noun count
# ---
# YOUR CODE GOES BELOW
#
# First, we will download the punkt and the averaged_perceptron_tagger into our notebook environment. 
# which will allow us to find the part of speech tags.
# ---
#
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# We create the function to check and get the part of speech tag count of a words in a given sentence
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [25]:
# Noun Count
# ---
# YOUR CODE GOES BELOW
#
df['noun_count'] = df.text.apply(lambda x: pos_check(x, 'noun'))
df.head()

,target,text,tweet_length,word_count,word_density,noun_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8


In [26]:
# Feature Construction: Verb count
# ---
# YOUR CODE GOES BELOW
#
df['verb_count'] = df.text.apply(lambda x: pos_check(x, 'verb'))
df.head()

,target,text,tweet_length,word_count,word_density,noun_count,verb_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5


In [27]:
# Feature Construction: Adjective count / Tweet
# ---
# YOUR CODE GOES BELOW
#
df['adj_count'] = df.text.apply(lambda x: pos_check(x, 'adj'))
df.head()


,target,text,tweet_length,word_count,word_density,noun_count,verb_count,adj_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2,1
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3,2
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2,0
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1,1
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5,0


In [28]:
# Feature Construction: Adverb count / Tweet
# ---
# YOUR CODE GOES BELOW
#
df['adv_count'] = df.text.apply(lambda x: pos_check(x, 'adv'))
df.head()

,target,text,tweet_length,word_count,word_density,noun_count,verb_count,adj_count,adv_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2,1,2
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3,2,1
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2,0,0
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1,1,1
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5,0,1


In [29]:
# Feature Construction: Pronoun 
# ---
# YOUR CODE GOES BELOW
#

df['pron_count'] = df.text.apply(lambda x: pos_check(x, 'pron'))
df.head()

,target,text,tweet_length,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2,1,2,0
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3,2,1,0
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2,0,0,0
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1,1,1,0
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5,0,1,0


In [30]:
# Feature Construction: Subjectivity
# ---
# YOUR CODE GOES BELOW
# 
# Function to get Subjectivity of text using the module textblob
def get_subjectivity(text):
    try:
        textblob = TextBlob(unicode(text, 'utf-8'))
        subj = textblob.sentiment.subjectivity
    except:
        subj = 0.0
    return subj

df['subjectivity'] = df.text.apply(get_subjectivity)
print(df['subjectivity'].value_counts())
df.head()

0.0    10000
Name: subjectivity, dtype: int64


,target,text,tweet_length,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count,subjectivity
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2,1,2,0,0.0
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3,2,1,0,0.0
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2,0,0,0,0.0
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1,1,1,0,0.0
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5,0,1,0,0.0


In [31]:
# Feature Construction: Polarity
# ---
# YOUR CODE GOES BELOW
# 
# Function to get polarity of text using the module textblob
def get_polarity(text):
    try:
        textblob = TextBlob(unicode(text, 'utf-8'))
        pol = textblob.sentiment.polarity
    except:
        pol = 0.0
    return pol

df['polarity'] = df.text.apply(get_polarity)
print(df['polarity'].value_counts())
df.head()

0.0    10000
Name: polarity, dtype: int64


,target,text,tweet_length,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count,subjectivity,polarity
0,0,obama forge muslim alliance civilized world didnt even drop cup tea,67,11,5.181818,6,2,1,2,0,0.0,0.0
1,4,spectacular prom ever bed serenading must answer sweet dream friend wonderful day,81,12,5.833333,5,3,2,1,0,0.0,0.0
2,0,overwhelmed today taking moment eat pray,40,6,5.833333,4,2,0,0,0,0.0,0.0
3,0,lin dork tres sad totally max fan sytycd,40,8,4.125000,5,1,1,1,0,0.0,0.0
4,0,crap counting hour dad could come home amp help watch son said going dinner first,81,15,4.466667,8,5,0,1,0,0.0,0.0


In [32]:
# Feature Construction: Word Level N-Gram TF-IDF Feature 


# ---
# YOUR CODE GOES BELOW
#
# Importing the TfidfVectorizer which will help us with this process
# ---
#
from sklearn.feature_extraction.text import TfidfVectorizer
#

tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', ngram_range=(1,3),  stop_words= 'english')
arr_word_vect = tfidf.fit_transform(df['text'])

# Show feature matrix / Priviewing the created sparse matrix
#
arr_word_vect.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
# Show tf-idf feature matrix
# ---
#
tfidf.get_feature_names()[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00', '09', '10', '100', '11', '12', '13', '15', '16', '17']

In [34]:
# Creating data frame to view our matrix
# ---
#
df_word_vect = pd.DataFrame(arr_word_vect.toarray(), columns=tfidf.get_feature_names())
print(df_word_vect.shape)
df_word_vect.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(10000, 1000)


,00,09,10,100,11,12,13,15,16,17,19,20,2009,21,22,24,30,40,50,aaa,aaaaaa,able,account,actually,ad,adam,add,added,afternoon,age,ago,agree,ah,aha,ahh,ahh hh,air,airport,al,album,alex,alright,amazing,amp,andy,angel,anna,annoying,answer,anymore,apparently,apple,arent,arm,art,article,ashley,ask,asleep,ate,aw,awake,award,away,awesome,ay,baby,bad,bag,ball,band,bar,bb,beach,bear,beat,beautiful,bed,bee,beer,believe,ben,best,best friend,better,big,bike,bird,birthday,bit,bitch,black,blog,bloody,blue,body,boo,book,bored,boring,bos,bought,bout,box,boy,brain,break,breakfast,bring,broke,broken,brother,brown,btw,bug,burn,bus,business,busy,buy,bye,ca,cake,cal,cali,called,came,camera,camp,cancelled,car,card,care,case,cat,catch,cause,cd,ch,chance,change,chat,check,cheer,chi,chill,chocolate,chris,church,city,ck,class,clean,close,club,code,coffee,cold,college,com,come,coming,comment,company,completely,computer,concert,congrats,cook,cool,country,couple,course,crap,crazy,cream,cried,cup,cut,cute,cyrus,da,dad,damn,dance,danny,date,david,day,dc,dd,dd lovato,dead,deal,dear,decided,deck,def,definitely,demi,design,didnt,die,died,dinner,disappointed,dj,dk,dm,doctor,doesnt,dog,donnie,dont,dont know,dont like,dont think,dont wanna,dont want,double,dr,dream,dress,drink,drinking,drive,driving,drop,dude,earlier,early,eat,eating,ed,ee,eee,eeee,eeeeee,eh,el,em,email,en,end,english,enjoy,enjoying,episode,er,evening,exactly,exam,excited,exhausted,eye,fa,face,facebook,fail,fair,fall,family,fan,fantastic,far,fast,fat,favorite,fb,feel,feel bad,feel better,feel like,feeling,fell,felt,fever,ff,figure,file,final,finally,fine,finger,finish,finished,fish,flight,flu,fly,fo,follow,follow friday,follower,following,food,foot,forever,forget,forgot,forward,free,french,fresh,friday,friend,fuck,fucking,fun,funny,future,game,garden,gave,gay,germany,getting,getting ready,gg,gh,gift,gig,girl,glad,glass,god,going,going bed,gone,gonna,good,good day,good luck,good morning,good night,good time,goodnight,google,gorgeous,got,got home,gotta,gr,graduation,grandma,great,great day,green,group,gt,guess,guy,gym,ha,ha ha,ha hah,hah,hah aha,hah hah,hair,half,hand,hang,hannah,happen,happened,happy,happy birthday,happy mother,happy mother day,hard,hat,hate,havent,hay,head,headache,heading,hear,heard,heart,hell,hello,help,hey,hh,hh hh,hi,high,hill,hit,hm,hmm,hmmm,ho,holiday,home,homework,honey,hoo,hope,hopefully,hoping,horrible,hot,hour,house,hr,hubby,hug,huge,huh,hun,hungry,hurt,husband,iam,ian,ice,ice cream,id,idea,ii,iii,iii iii,iii iii iii,ill,im,im getting,im glad,im going,im gonna,im good,im hungry,im really,im sick,im sorry,im sure,info,ing,inside,instead,interesting,internet,iphone,ipod,ish,isnt,itll,ive,ive got,jack,jam,james,jason,jay,jealous,jess,job,joe,john,join,jon,jonas,jonas brother,jonathan,jordan,joy,june,jus,ka,kate,katie,kay,kelly,key,kick,kid,kill,killing,kind,kinda,king,kiss,kitty,knee,knew,knight,know,know im,la,lady,lame,land,laptop,late,later,laugh,lauren,lazy,le,learn,leave,leaving,lee,left,leg,let,let know,ley,li,life,light,like,lil,line,link,list,listen,listening,little,live,lo,load,lol,london,lonely,long,long day,long time,loo,look,look like,looked,looking,looking forward,lost,lot,lovato,love,loved,lovely,low,lt,lu,luck,lucky,lunch,luv,ly,mac,mad,mail,make,making,mall,man,mao,mar,mark,math,matt,matter,maybe,mcfly,mean,meant,medium,meet,meeting,men,mess,message,mi,michael,mike,mile,miley,miley cyrus,min,mind,minute,miss,missed,missing,mm,mmm,mo,mom,moment,mommy,monday,money,month,mood,moon,morning,mother,mother day,movie,moving,mr,mtv,mu,mum,music,myspace,na,nah,nap,nd,ne,near,need,needed,net,new,news,ng,nice,nice day,nick,night,nite,nope,nose,note,nt,number,ny,nyc,office,official,officially,oh,oh god,ok,okay,ol,old,omg,online,oo,ooo,ooo ooo,open,order,outside,oxo,oz,pa,packing,page,pain,paper,paris,park,party,past,pay,pc,people,perfect,person,ph,phone,photo,pic,pick,picture,pink,place,plan,plane,play,played,playing,pm,point,pool,poor,post,posted,power,ppl,pr,pre,pretty,pr

In [35]:
# Feature Construction: Character Level N-Gram TF-IDF Feature
# ---
# YOUR CODE GOES BELOW
# 

tfidf_char = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='char', ngram_range=(1,3),  stop_words= 'english')
arr_char_vect = tfidf_char.fit_transform(df['text'])
arr_char_vect.toarray()


array([[0.2442019 , 0.        , 0.        , ..., 0.        , 0.08323056,
        0.        ],
       [0.23113135, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.17551132, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.21992303, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18421584, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.28106559, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [36]:
# Show tf-idf feature matrix
# ---
#
tfidf_char.get_feature_names()[:10]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[' ', ' 1', ' 1 ', ' 2', ' 2 ', ' 3', ' 3 ', ' 4', ' 4 ', ' a']

In [37]:
# Creating data frame to view our matrix
# ---
#
df_char_vect = pd.DataFrame(arr_char_vect.toarray(), columns=tfidf_char.get_feature_names())
print(df_char_vect.shape)
df_char_vect.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(10000, 1000)


,,1,1,2,2,3,3,4,4,a,ac,al,am,an,ar,aw,b,b,ba,be,bi,bl,bo,br,bu,c,ca,ch,cl,co,cr,cu,d,da,de,di,do,dr,e,e,ea,en,ev,ex,f,fa,fe,fi,fo,fr,fu,g,ga,ge,gi,go,gr,gu,h,ha,he,hi,ho,hu,i,il,im,in,j,jo,k,ke,ki,kn,l,la,le,li,lo,lu,m,ma,me,mi,mo,mu,n,na,ne,ni,no,o,oh,on,oo,p,pa,pe,pi,pl,po,pr,pu,q,qu,r,ra,re,ri,ro,s,sa,sc,se,sh,si,sl,so,sp,st,su,t,ta,te,th,ti,to,tr,tw,u,u,un,v,vi,w,wa,we,wh,wi,wo,x,y,ya,ye,yo,z,0,0,1,1,2,2,3,3,4,4,5,5,6,7,8,9,a,a,a g,aa,ab,ac,ace,ach,ack,ad,ad,ady,ag,age,ah,ah,ai,ail,ain,air,ait,ak,ake,al,al,all,am,am,ame,amp,an,an,anc,and,ang,ank,ann,ant,any,ap,app,ar,ar,ard,are,art,as,ase,ass,ast,at,at,atc,ate,ath,ati,ats,au,av,ave,aw,ay,ay,az,b,b,ba,bac,bad,be,bea,ber,bi,bl,ble,bo,boo,br,bro,bu,by,c,c,ca,cal,can,car,ce,ce,ch,ch,cha,che,chi,cho,ci,ck,ck,cke,cl,co,col,com,con,cou,cr,cra,ct,cu,d,d,d a,d b,d c,d d,d e,d f,d g,d h,d l,d m,d n,d p,d r,d s,d t,d w,da,day,dd,de,de,der,di,did,din,dn,dnt,do,don,dr,du,dy,dy,e,e,e a,e b,e c,e d,e e,e f,e g,e h,e i,e l,e m,e n,e o,e p,e r,e s,e t,e u,e w,ea,ead,eal,ear,eas,eat,eb,ec,ed,ed,ee,ee,eed,eek,eel,eep,eet,ef,eg,ei,ek,el,el,eli,ell,ely,em,en,en,end,ent,eo,ep,ep,er,er,ere,eri,ery,es,es,ess,est,et,et,ett,ev,eve,ew,ew,ex,ey,ey,f,f,fa,fe,fee,ff,fi,fin,fl,fo,fol,for,fr,fri,ft,fu,ful,fun,g,g,g a,g b,g c,g d,g f,g g,g h,g l,g m,g n,g p,g r,g s,g t,g w,ga,ge,ge,get,gh,gh,ght,gi,gir,gl,go,go,goi,gon,goo,got,gr,gra,gre,gu,h,h,h c,h h,h s,h t,h w,ha,ha,hah,han,hap,har,hat,he,hea,hel,her,hey,hh,hi,hin,ho,hom,hon,hoo,hop,hot,hou,how,hr,ht,ht,hu,i,i,ia,ic,ic,ice,ick,id,id,ide,idn,ie,ie,ien,if,ig,igh,ik,ike,il,il,ile,ill,im,im,ime,in,in,ina,ind,ine,ing,ini,ink,int,io,ion,ip,ir,ire,is,ish,iss,ist,it,it,ite,itt,iv,ive,j,j,ja,je,jo,k,k,k c,k l,k s,k t,ka,ke,ke,ked,ken,ki,kin,kn,kno,ks,ks,l,l,l b,l c,l g,l l,l m,l s,l t,l w,la,lan,las,lat,lay,ld,ld,le,le,lea,lee,let,li,lik,lin,lis,lk,ll,ll,lle,lli,llo,lly,lo,lol,lon,loo,lov,low,lt,lt,lu,ly,ly,m,m,m g,m s,ma,mak,man,mar,mb,me,me,men,mer,mi,mil,min,mis,mm,mo,mon,mor,mp,mp,mu,muc,mus,n,n,n b,n c,n g,n l,n m,n s,n t,n w,na,na,nal,nc,nce,nd,nd,nda,nde,ne,ne,nee,new,ng,ng,nge,ni,nic,nig,nin,nk,nk,nks,nn,nna,no,not,now,ns,nt,nt,nte,ny,ny,o,o,o o,o s,oa,ob,oc,od,od,oda,oe,of,og,oh,oh,oi,oin,ok,ok,oke,ol,ol,old,oll,om,om,ome,omo,on,on,ond,one,ong,oni,ont,oo,oo,ood,ook,ool,oon,ooo,op,op,ope,or,or,ore,ork,orn,orr,ort,os,ost,ot,ot,oth,ou,oug,oul,oun,our,ous,out,ov,ove,ow,ow,oy,p,p,pa,par,pe,pe,pen,per,ph,pho,pi,pic,pin,pl,pla,ple,po,pp,ppy,pr,pre,pro,pu,py,py,q,qu,quo,r,r,r a,r b,r c,r d,r f,r g,r h,r l,r m,r s,r t,r w,ra,rai,ran,rd,rd,re,re,rea,red,ree,ren,res,rg,ri,rie,rig,rin,rk,rk,rl,rm,rn,rni,ro,rou,row,rr,rro,rry,rs,rt,rt,ru,ry,ry,s,s,s s,sa,sad,sc,se,se,sed,see,sh,sh,sha,shi,sho,si,sic,sin,sk,sl,sle,sn,snt,so,som,son,sor,sp,ss,ss,sse,ssi,st,st,sta,ste,sti,sto,stu,su,sun,sur,sw,t,t,t a,t b,t c,t d,t e,t f,t g,t h,t i,t k,t l,t m,t n,t p,t r,t s,t t,t w,ta,tal,tar,tc,tch,te,te,ted,tel,ten,ter,th,th,tha,the,thi,tho,ti,tic,til,tim,tin,tio,tl,to,tod,tom,ton,tor,tr,tra,try,ts,ts,tt,tte,tti,tu,tur,tw,twe,twi,ty,ty,u,u,ua,ub,uc,uch,uck,ud,ue,ug,ugh,ui,ul,uld,um,un,un,und,uo,uo,up,ur,ur,ure,us,us,use,usi,ut,ut,v,v,va,ve,ve,ven,ver,vi,vin,vo,w,w,wa,wai,wan,war,wat,way,we,wee,wel,wh,wi,win,wis,wit,wo,wor,wou,x,x,xt,y,y,y a,y b,y c,y d,y e,y f,y g,y h,y i,y l,y m,y n,y p,y r,y s,y t,y w,ya,ye,yea,yes,yi,yin,yo,you,ys,z,z
0,0.244202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054803,0.0,0.095201,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.097931,0.0,0.0,0.0,0.000000,0.0,0.117072,0.106921,0.000000,0.0,0.093786,0.000000,0.109338,0.062906,0.0,0.000000,0.0,0.099259,0.0,0.054941,0.000000,0.0,0.00000,0.096368,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051134,0.000000,0.0,0.0,0.000000,0.098050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0

In [38]:
# Let's prepare the constructed features for modeling
# ---
#
X_metadata = np.array(df.iloc[:, 2:12])
X_metadata

array([[67.        , 11.        ,  5.18181818, ...,  0.        ,
         0.        ,  0.        ],
       [81.        , 12.        ,  5.83333333, ...,  0.        ,
         0.        ,  0.        ],
       [40.        ,  6.        ,  5.83333333, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [45.        ,  8.        ,  4.75      , ...,  0.        ,
         0.        ,  0.        ],
       [34.        ,  6.        ,  4.83333333, ...,  0.        ,
         0.        ,  0.        ],
       [44.        , 10.        ,  3.5       , ...,  0.        ,
         0.        ,  0.        ]])

In [39]:
# We combine our two tfidf (sparse) matrices and X_metadata
# ---
#
X = scipy.sparse.hstack([arr_word_vect, arr_char_vect,  X_metadata])
X

<10000x2010 sparse matrix of type '<class 'numpy.float64'>'
	with 938102 stored elements in COOrdinate format>

In [40]:
# Getting our response variable
# ---
#
y = np.array(df.iloc[:, 0])
y

array([0, 4, 0, ..., 0, 4, 0])

### 4. Data Modelling

During this step, we will use machine learning algorithms to train and test our sentiment analysis models.

In [41]:
# Splitting our data
# ---
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
# Fitting our model
# ---
#

# Importing the algorithms
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

nb_classifier = MultinomialNB() 
lr_classifier = LogisticRegression(max_iter=1000) 

# Training our model
nb_classifier.fit(X_train, y_train) 
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [43]:
# Making predictions
# ---
#
y_predict_nb = nb_classifier.predict(X_test) 
y_predict_lr = lr_classifier.predict(X_test)

In [44]:
# Evaluating the Models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Accuracy scores
# ---
#
print("Naive Bayes Classifier:\n", accuracy_score(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", accuracy_score(y_test, y_predict_lr))

Naive Bayes Classifier:
 0.7285
Logistic Regression Classifier: 
 0.734


In [45]:
# Confusion matrices
# ---
# 
print("Naive Bayes Classifier: \n", confusion_matrix(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", confusion_matrix(y_test, y_predict_lr))

Naive Bayes Classifier: 
 [[762 288]
 [255 695]]
Logistic Regression Classifier: 
 [[765 285]
 [247 703]]


In [46]:
# Classification Reports
# ---
#
print("Naive Bayes Classifier: \n", classification_report(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", classification_report(y_test, y_predict_lr))

Naive Bayes Classifier: 
               precision    recall  f1-score   support

           0       0.75      0.73      0.74      1050
           4       0.71      0.73      0.72       950

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000

Logistic Regression Classifier: 
               precision    recall  f1-score   support

           0       0.76      0.73      0.74      1050
           4       0.71      0.74      0.73       950

    accuracy                           0.73      2000
   macro avg       0.73      0.73      0.73      2000
weighted avg       0.73      0.73      0.73      2000



**Evaluation our Models**

* **Accuracy:** the percentage of texts that were assigned the correct topic.
* **Precision:** the percentage of texts the classifier classified correctly out of the total number of texts it predicted for each topic
* **Recall:** the percentage of texts the model predicted for each topic out of the total number of texts it should have predicted for that topic.
* **F1 Score:** the average of both precision and recall.

To improve our model, we can try perfoming other text processing techniques that would better prepare our data for fitting our model. We can also use different vectorizing techniques, implement other machine learning models and perform hyperparameter tuning.

### 5. Recommendations


Our best model had an accuracy of 73.25% and use it for classifying newer tweets. We can improve this performance by performing hyperparameter tuning and feature engineering methods. 